Diving Deeper into Weights & Biases
In this notebook, we will explore the following

Versioning datasets using Artifacts.
Exploring and visualizing our datasets with Tables.
Baseline Experiment with a Random Forest Classification Model.

In [1]:
import os
import pickle

import wandb
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [9]:
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", job_type="log_data")

# Log the `data` directory as an artifact
artifact = wandb.Artifact('Titanic', type='dataset', metadata={"Source": "https://www.kaggle.com/competitions/titanic/data"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

# End the WandB Run
wandb.finish()

wandb: Adding directory to artifact (./data)... Done. 0.0s


## Versioning the data

In [1]:
import wandb
run = wandb.init(project="mlops-zoomcamp-wandb", job_type="log_data")
artifact = run.use_artifact('sotoblanco/mlops-zoomcamp-wandb/Titanic:v0', type='dataset')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sotoblanco. Use `wandb login --relogin` to force relogin


wandb:   4 of 4 files downloaded.  


Read the dataset files

In [2]:
train_df = pd.read_csv(os.path.join(artifact_dir, 'train.csv'))

NameError: name 'pd' is not defined

In [16]:
num_train_examples = int(0.8 * len(train_df))
num_val_examples = len(train_df) - num_train_examples

print(num_train_examples, num_val_examples)

712 179


In [17]:
train_df["Split"] = ["Train"] * num_train_examples + ["Val"] * num_val_examples
train_df.to_csv("data/train.csv", encoding='utf-8', index=False)

In [18]:
# Log the 'data' directory as an artifact
artifact = wandb.Artifact('Titanic', type='dataset',
                           metadata={'Source': "https://www.kaggle.com/competitions/titanic/data"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

# end the wandB Run
wandb.finish()

wandb: Adding directory to artifact (./data)... Done. 0.0s


## Explore the dataset

In [19]:
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", job_type="explore_data")

# Fetch the latest version of the dataset artifact 
artifact = wandb.use_artifact('sotoblanco/mlops-zoomcamp-wandb/Titanic:latest', type='dataset')
artifact_dir = artifact.download()

# Read the files
train_val_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))

wandb:   4 of 4 files downloaded.  


In [20]:
# create table corresponding to datasets
train_val_table = wandb.Table(dataframe=train_df)

# log the table to the W&B
wandb.log({"train-val-able": train_val_table})

wandb.finish()

## Fit a baseline model

In [4]:
import wandb
import pandas as pd
import os
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", name="baseline_experiment-2", job_type="train")

# Fetch the latest version of the dataset artifact 
artifact = wandb.use_artifact('sotoblanco/mlops-zoomcamp-wandb/Titanic:latest', type='dataset')
artifact_dir = artifact.download()

# Read the files
train_val_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
#test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))

wandb:   4 of 4 files downloaded.  


In [5]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X_train = pd.get_dummies(train_val_df[features][train_val_df["Split"] == "Train"])
X_val = pd.get_dummies(train_val_df[features][train_val_df["Split"] == "Validation"])
y_train = train_val_df["Survived"][train_val_df["Split"] == "Train"]
y_val = train_val_df["Survived"][train_val_df["Split"] == "Validation"]

In [8]:
# import randomforestclassifier
from sklearn.ensemble import RandomForestClassifier

model_params = {"n_estimators": 100, "max_depth": 10, "random_state": 1}
wandb.config = model_params

model = RandomForestClassifier(**model_params)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_probas_train = model.predict_proba(X_train)
y_pred_val = model.predict(X_val)
y_probas_val = model.predict_proba(X_val)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Sex_female
- Sex_male


In [27]:
X_val

,Pclass,SibSp,Parch


In [ ]:
X_train

In [ ]:
wandb.log({
    "Train/Accuracy": accuracy_score(y_train, y_pred_train),
    "Validation/Accuracy": accuracy_score(y_val, y_pred_val),
    "Train/Presicion": precision_score(y_train, y_pred_train),
    "Validation/Presicion": precision_score(y_val, y_pred_val),
    "Train/Recall": recall_score(y_train, y_pred_train),
    "Validation/Recall": recall_score(y_val, y_pred_val),
    "Train/F1-Score": f1_score(y_train, y_pred_train),
    "Validation/F1-Score": f1_score(y_val, y_pred_val),
})

In [ ]:
label_names = ["Not-Survived", "Survived"]

wandb.sklearn.plot_class_proportions(y_train, y_val, label_names)
wandb.sklearn.plot_summary_metrics(model, X_train, y_train, X_val, y_val)
wandb.sklearn.plot_roc(y_val, y_probas_val, labels=label_names)
wandb.sklearn.plot_precision_recall(y_val, y_probas_val, labels=label_names)
wandb.sklearn.plot_confusion_matrix(y_val, y_pred_val, labels=label_names)

In [ ]:
# Save your model
with open("random_forest_classifier.pkl", "wb") as f:
    pickle.dump(model, f)

# Log your model as a versioned file to Weights & Biases Artifact
artifact = wandb.Artifact(f"titanic-random-forest-model", type="model")
artifact.add_file("random_forest_classifier.pkl")
wandb.log_artifact(artifact)


# End the WandB Run
wandb.finish()